In [9]:
import pandas as pd

In [10]:
mjgross = pd.read_csv('D:/Flatiron/Project/Zips/bom.movie_gross.csv/bom.movie_gross.csv')
imdbname = pd.read_csv('D:/Flatiron/Project/Zips/imdb.name.basics.csv/name.basics.csv')
imdbtitle = pd.read_csv('D:/Flatiron/Project/Zips/imdb.title.akas.csv/title.akas.csv')
imdbtitle2 = pd.read_csv('D:/Flatiron/Project/Zips/imdb.title.basics.csv/title.basics.csv')
imdbcrew = pd.read_csv('D:/Flatiron/Project/Zips/imdb.title.crew.csv/title.crew.csv')
imdbprinc = pd.read_csv('D:/Flatiron/Project/Zips/imdb.title.principals.csv/title.principals.csv')
imdbrate = pd.read_csv('D:/Flatiron/Project/Zips/imdb.title.ratings.csv/title.ratings.csv')
rtinfo = pd.read_csv('D:/Flatiron/Project/Zips/rt.movie_info.tsv/rt.movie_info.tsv', delimiter='\t')
rtreviews = pd.read_csv('D:/Flatiron/Project/Zips/rt.reviews.tsv/rt.reviews.tsv', sep=None)
tmdbmovies = pd.read_csv('D:/Flatiron/Project/Zips/tmdb.movies.csv/tmdb.movies.csv')
tnbudgets = pd.read_csv('D:/Flatiron/Project/Zips/tn.movie_budgets.csv/tn.movie_budgets.csv')

In [11]:
# joined imdbcrew and imdbtitle2
imdbcrewtitle2 = imdbtitle2.join(imdbcrew.set_index('tconst'), on='tconst')


In [12]:
#Editing, dropping mostly empty or useless columns
imdbname.drop(['birth_year','death_year'], axis=1, inplace=True)
imdbtitle.drop(['attributes','language'], axis=1, inplace=True)
imdbprinc.drop(['job','characters'], axis=1, inplace=True)
rtinfo.drop(['synopsis','theater_date','dvd_date','studio','currency'], axis=1, inplace=True)

In [13]:
#Just to have a handy reference point
print('\n imdbname \n'); print(len(imdbname)); print(imdbname.isna().sum())
print('\n imdbtitle \n'); print(len(imdbtitle)); print(imdbtitle.isna().sum())
print('\n imdbcrewtitle2 \n'); print(len(imdbcrewtitle2)); print(imdbcrewtitle2.isna().sum())
print('\n imdbprinc \n'); print(len(imdbprinc)); print(imdbprinc.isna().sum())
print('\n imdbrate \n'); print(len(imdbrate)); print(imdbrate.isna().sum())
print('\n rtinfo \n'); print(len(rtinfo)); print(rtinfo.isna().sum())
print('\n rtreviews \n'); print(len(rtreviews)); print(rtreviews.isna().sum())
print('\n tmdbmovies \n'); print(len(tmdbmovies)); print(tmdbmovies.isna().sum())
print('\n tnbudgets \n'); print(len(tnbudgets)); print(tnbudgets.isna().sum())


 imdbname 

606648
nconst                    0
primary_name              0
primary_profession    51340
known_for_titles      30204
dtype: int64

 imdbtitle 

331703
title_id                  0
ordering                  0
title                     0
region                53293
types                163256
is_original_title        25
dtype: int64

 imdbcrewtitle2 

146144
tconst                 0
primary_title          0
original_title        21
start_year             0
runtime_minutes    31739
genres              5408
directors           5727
writers            35883
dtype: int64

 imdbprinc 

1028186
tconst             0
ordering           0
nconst             0
category           0
characters    634826
dtype: int64

 imdbrate 

73856
tconst           0
averagerating    0
numvotes         0
dtype: int64

 rtinfo 

1560
id               0
rating           3
genre            8
director       199
writer         449
box_office    1220
runtime         30
dtype: int64

 rtreviews 

54432
id 

In [ ]:
#in progress DON'T RUN
def splitmult(df, column):
    for i, item in enumerate(df[column]):
        each = item.split(sep=',')
        item = each[0]
        for e in range(1, len(each)):
            new_rows = {}
            new_rows.append(str(df.iloc[i].loc[column]) + ' : ' + )

In [14]:
        for e in each:
            new_rows = {}
            for i2 in range(len(df.columns)):
                new_rows.append(str(df.columns[i2]) + ' : ' + str(df.loc[column][i]))

6

In [25]:
imdbname['primary_name'][1]

'Joseph Bauer'

In [ ]:
for name in 

In [32]:
imdbprinc

,tconst,ordering,nconst,category,characters
0,tt0111414,1,nm0246005,actor,"[""The Man""]"
1,tt0111414,2,nm0398271,director,NaN
2,tt0111414,3,nm3739909,producer,NaN
3,tt0323808,10,nm0059247,editor,NaN
4,tt0323808,1,nm3579312,actress,"[""Beth Boothby""]"
...,...,...,...,...,...
1028181,tt9692684,1,nm0186469,actor,"[""Ebenezer Scrooge""]"
1028182,tt9692684,2,nm4929530,self,"[""Herself"",""Regan""]"
1028183,tt9692684,3,nm10441594,director,NaN
1028184,tt9692684,4,nm6009913,writer,NaN


In [33]:
imdbname

,nconst,primary_name,primary_profession,known_for_titles
0,nm0061671,Mary Ellen Bauder,"miscellaneous,production_manager,producer","tt0837562,tt2398241,tt0844471,tt0118553"
1,nm0061865,Joseph Bauer,"composer,music_department,sound_department","tt0896534,tt6791238,tt0287072,tt1682940"
2,nm0062070,Bruce Baum,"miscellaneous,actor,writer","tt1470654,tt0363631,tt0104030,tt0102898"
3,nm0062195,Axel Baumann,"camera_department,cinematographer,art_department","tt0114371,tt2004304,tt1618448,tt1224387"
4,nm0062798,Pete Baxter,"production_designer,art_department,set_decorator","tt0452644,tt0452692,tt3458030,tt2178256"
...,...,...,...,...
606643,nm9990381,Susan Grobes,actress,NaN
606644,nm9990690,Joo Yeon So,actress,"tt9090932,tt8737130"
606645,nm9991320,Madeline Smith,actress,"tt8734436,tt9615610"
606646,nm9991786,Michelle Modigliani,producer,NaN


In [34]:
imdbtitle

,title_id,ordering,title,region,types,is_original_title
0,tt0369610,10,Джурасик свят,BG,NaN,0.0
1,tt0369610,11,Jurashikku warudo,JP,imdbDisplay,0.0
2,tt0369610,12,Jurassic World: O Mundo dos Dinossauros,BR,imdbDisplay,0.0
3,tt0369610,13,O Mundo dos Dinossauros,BR,NaN,0.0
4,tt0369610,14,Jurassic World,FR,imdbDisplay,0.0
...,...,...,...,...,...,...
331698,tt9827784,2,Sayonara kuchibiru,NaN,original,1.0
331699,tt9827784,3,Farewell Song,XWW,imdbDisplay,0.0
331700,tt9880178,1,La atención,NaN,original,1.0
331701,tt9880178,2,La atención,ES,NaN,0.0


In [37]:
# idea: make dictionary, keys are nm#####s, values are names. use replace function

In [57]:
#dictionary to replace names
names = []
for n in imdbname['nconst']:
    names.append(n)
len(names)
real_names = []
for n in imdbname['primary_name']:
    real_names.append(n)
len(real_names)
name_dict = dict(zip(names, real_names))

In [83]:
# def replcnm():
#     for i, item in enumerate(imdbprinc.loc[:,'nconst']):
#         if str(item) in names:
#             imdbprinc.replace(str(item), name_dict[str(item)])
        # try:
        #     imdbprinc.iloc[i].loc['nconst'].replace(item, name_dict[item])
        # except:
        #     continue

    #     if item not in names:
    #         continue
    #     if item in names:
    #         imdbprinc.replace(item, name_dict[item])
newplc = imdbprinc.nconst.apply(imdbprinc.replace(name_dict))
newplc

KeyboardInterrupt: 

In [77]:
name_dict['nm8479383']

'Milan Pjevac'

In [ ]:
newplc

In [64]:
name_dict.keys()

533', 'nm8479383', 'nm8481025', 'nm8481580', 'nm8482785', 'nm8484828', 'nm8485439', 'nm8485659', 'nm8485840', 'nm8485868', 'nm8486204', 'nm8486512', 'nm8486584', 'nm8487511', 'nm8487525', 'nm8487754', 'nm8487755', 'nm8488065', 'nm8489505', 'nm8489852', 'nm8490559', 'nm8490691', 'nm8490845', 'nm8491526', 'nm8492358', 'nm8492359', 'nm8492793', 'nm8493403', 'nm8493404', 'nm8494478', 'nm8495875', 'nm8496327', 'nm8497578', 'nm8498311', 'nm8498422', 'nm8498549', 'nm8498550', 'nm8499149', 'nm8499401', 'nm8499608', 'nm8499611', 'nm8500007', 'nm8500256', 'nm8501113', 'nm8501259', 'nm8501390', 'nm8501481', 'nm8502003', 'nm8502497', 'nm8502687', 'nm8503761', 'nm8506228', 'nm8508865', 'nm8509686', 'nm8509699', 'nm8509816', 'nm8509866', 'nm8510398', 'nm8512008', 'nm8512340', 'nm8512752', 'nm8513328', 'nm8516614', 'nm8517617', 'nm8517647', 'nm8518307', 'nm8518416', 'nm8518732', 'nm8518892', 'nm8519373', 'nm8519721', 'nm8520623', 'nm8521559', 'nm8521908', 'nm8523788', 'nm8526044', 'nm8526763', 'nm852